##This notebook uses gradient class activation mapping to determine which pixels are more important for model classifications

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/'

Read in data. This may take a few minutes, especially if there are thousands of images.

In [ ]:
fns = get_image_files(path)
fns

(#4229) [Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_c228dd99-0d0c-4b5f-907f-7e6db29189d7.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_c74d3958-d47e-431b-8d2f-9adde3de3abd.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_cba87a33-53d1-4818-8793-a362513c8a94.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_cce75b07-dae8-4246-ae26-dab581839cc3.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_d171172d-46c3-4487-9077-e0048b7870e2.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_e10f584a-87eb-46c2-b6a8-3ebed026b445.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_e14340ea-0296-4e00-98d2-d804933a38c0.jpg'),Path('/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1/SC_trim1/cerasoides_e6ee68a7-b6e4-4f78-89d5-d478fbb643

Will need to make sure these parameters (or at least the dls) match the model made and imported

---



In [ ]:
pilot_cp_nuc = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = pilot_cp_nuc.dataloaders(path)

In [ ]:
pilot_cp_nuc = pilot_cp_nuc.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())


dls = pilot_cp_nuc.dataloaders(path)

In [ ]:
cd /content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1

/content/drive/MyDrive/DL_images/CP_NUC/model_dev_trim1


The three items in this directory: two subdirectories and the model, saved as a .pkl file

In [ ]:
ls

prunus_res50_cpnuc_v5_trim1_fp16_24epochs.pkl  SC_trim1/  TR_trim1/


In [ ]:
learn_inf = load_learner('prunus_res50_cpnuc_v5_trim1_fp16_24epochs.pkl')


In [ ]:
learn_inf.model.cuda()

#Now define hooks to be used for gradient CAM

In [ ]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()


In [ ]:
cd /content/drive/MyDrive/DL_images/CP_NUC/

/content/drive/MyDrive/DL_images/CP_NUC


In [ ]:
import pandas as pd

## First, look at images that are from temperate racemose species that were classified as solitary/corymbose

```
# This is formatted as code
```



In [ ]:
hold_img = pd.read_csv("temp_racemose_probs_trim1_v5_sc233.csv")

In [ ]:
cd "/content/drive/MyDrive/DL_images/final_nbs/cp_nuc_models/trim/temp_racemose_padus_trim1"

/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1


In [ ]:
learn_inf.dls.vocab

['SC_trim1', 'TR_trim1']

In [ ]:
hold_img

,image,classification,probability
0,serotina_062639c7-effa-41c0-a209-d7a5cc7ef83e.jpg,SC_trim1,1.0000
1,serotina_060eab75-9b9a-47ab-a0d8-a95519c4be13.jpg,SC_trim1,1.0000
2,serotina_06517809-091a-45f8-867c-d24302188732.jpg,SC_trim1,1.0000
3,serotina_05b2cbee-0ae8-4958-b356-24d6505da560.jpg,SC_trim1,1.0000
4,serotina_05301f64-b5bc-4363-9b80-247741752d84.jpg,SC_trim1,1.0000
...,...,...,...
228,grayana_93f4e119-10f7-4f85-bc6c-a618fd321e0c.jpg,SC_trim1,0.9143
229,virginiana_02e6ff97-a182-47fb-861b-6cde071814b1.jpg,SC_trim1,0.9139
230,serotina_0452c818-8a2b-4fc5-905c-ba6608f90564.jpg,SC_trim1,0.9126
231,virginiana_018b861d-aa43-48a1-aff1-63dab210380c.jpg,SC_trim1,0.9123


This loop uses hooks to get features from the last model layer

In [ ]:
for counter in range(0,234):
 # counter=0;
  img = hold_img['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[0]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
      #  cls 0 for sc, cls 1 for trop race
  cls = 0
  with HookBwd(learn_inf.model[0]) as hookg:
      with Hook(learn_inf.model[0]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_sc_lastlayer_' + str(counter) + '.png';
  fig.savefig(filename);
  plt.close(fig);

# The following loops access inner layers of the model

In [ ]:
for counter in range(0,234):
  img = hold_img['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 0
  with HookBwd(learn_inf.model[0][-2]) as hookg:
      with Hook(learn_inf.model[0][-2]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_sc_secondlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

In [ ]:
for counter in range(0,239):
  img = hold_img['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 0
  with HookBwd(learn_inf.model[0][-3]) as hookg:
      with Hook(learn_inf.model[0][-3]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_sc_thirdlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

In [ ]:
for counter in range(0,239):
  img = hold_img['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 0
  with HookBwd(learn_inf.model[0][-4]) as hookg:
      with Hook(learn_inf.model[0][-4]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_sc_fourthlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

## Now load data for temperate racemose images classified as tropical racemose and get images for the last 4 layers

In [ ]:
cd "/content/drive/MyDrive/DL_images/CP_NUC/"

/content/drive/MyDrive/DL_images/CP_NUC


In [ ]:
hold_img2 = pd.read_csv("temp_racemose_probs_trim1_v5_tr194.csv")

In [ ]:
hold_img2

,image,classification,probability
0,virginiana_002510c4-c44f-473d-8839-a74ad1b5c8f8.jpg,TR_trim1,1.0000
1,serotina_05608d39-da63-4f9b-baa4-896c771f405f.jpg,TR_trim1,1.0000
2,serotina_048f5080-952a-40f3-883f-ce2704814542.jpg,TR_trim1,1.0000
3,serotina_04293b68-ddb4-4333-8e00-6ac75ff656af.jpg,TR_trim1,1.0000
4,serotina_0388a63b-2699-4334-b01e-91c9af5bb410.jpg,TR_trim1,1.0000
...,...,...,...
189,serotina_03f83330-c624-44b0-911d-231b7faaeaff.jpg,TR_trim1,0.9152
190,virginiana_0603b1a9-51a9-4514-ae7f-e81dab37585b.jpg,TR_trim1,0.9044
191,virginiana_04d4d4e4-dec0-4a15-bcaf-291422d8d405.jpg,TR_trim1,0.9034
192,grayana_04f6fbb3-1718-4c26-a1ef-3d6c19573ee2.jpg,TR_trim1,0.9033


In [ ]:
cd "/content/drive/MyDrive/DL_images/final_nbs/cp_nuc_models/trim/temp_racemose_padus_trim1"

/content/drive/MyDrive/DL_images/final_nbs/cp_nuc_models/trim/temp_racemose_padus_trim1


In [ ]:
for counter in range(0,194):
  img = hold_img2['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 1
  with HookBwd(learn_inf.model[0][-4]) as hookg:
      with Hook(learn_inf.model[0][-4]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_tr_fourthlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

In [ ]:
for counter in range(0,194):
  img = hold_img2['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 1
  with HookBwd(learn_inf.model[0][-3]) as hookg:
      with Hook(learn_inf.model[0][-3]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_tr_thirdlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

In [ ]:
for counter in range(0,194):
  img = hold_img2['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
        #  cls 0 for sc, cls 1 for trop race
  cls = 1
  with HookBwd(learn_inf.model[0][-2]) as hookg:
      with Hook(learn_inf.model[0][-2]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_tr_secondlastlayer_' + str(counter) + '.png'
  fig.savefig(filename)
  plt.close(fig)

In [ ]:
for counter in range(0,194):
 # counter=0;
  img = hold_img2['image'][counter];
  print(img);
  x, = first(dls.test_dl([img]));
  with Hook(learn_inf.model[0]) as hook:
      with torch.no_grad(): output = learn_inf.model.eval()(x.cuda())
      act = hook.stored
      #  cls 0 for sc, cls 1 for trop race
  cls = 1
  with HookBwd(learn_inf.model[0]) as hookg:
      with Hook(learn_inf.model[0]) as hook:
          output = learn_inf.model.eval()(x.cuda())
          act = hook.stored
      output[0,cls].backward()
      grad = hookg.stored
  w = grad[0].mean(dim=[1,2], keepdim=True)
  cam_map = (w * act[0]).sum(0)
  x_dec = TensorImage(dls.train.decode((x,))[0][0])
  fig,ax = plt.subplots()
  x_dec.show(ctx=ax)
  ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
                interpolation='bilinear', cmap='magma');
  filename = '/content/drive/MyDrive/DL_images/CP_NUC/temp_racemose_padus_CAM_trim1/output_tr_lastlayer_' + str(counter) + '.png';
  fig.savefig(filename);
  plt.close(fig)